In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import glob
from scipy import stats

plt.rc('font', family='Times New Roman', size=12) # change the font of all text
s_B = 3540 #[G], B value for the interal standard

# file folder, fill height and mass for the scans that will create the dose response curve (group 1)
group1_folder_path = ''
group1_mass = np.array([0.4306, 0.4519, 0.4282, 0.4552, 0.4263, 0.4424]) # g
group1_fill_height = np.array([57, 60, 58, 58, 57, 60]) # mm

# file folder, fill height and mass for the additional samples (group 2)
group2_folder_path = ''
group2_mass = np.array([0.4436, 0.44, 0.447, 0.4507, 0.4539, 0.4539, 0.4413]) # g
group2_fill_height = np.array([57, 58, 60, 60, 59, 58, 57]) # mm

# error on the mass and fill height
mass_err = 0.0001 # g
height_err = 1 # mm

The cell below defines the indices used to determine the central alanine peak P2P amplitude and the internal standard P2P amplitude. To verify the peak minimums and maximums have been identified correctly, the specta are plotted with a black bar at the minimum and maximum of both peaks. Should the bars be in the wrong location, adjust accordingly. 

Important note: the file naming convention used when writing this script was 'Alanine-[Dose] Gy-IS [Height]mm-Scan #.txt', if using a different format then changes will need to be made to line 9 in the cell below. 

In [ ]:
# define the central alanine peak and internal standard peak start and stop indexes
a_start = 410
a_stop = 570
s_start = 740
s_stop = 830
bar_width = 10

for file_path in glob.glob(group1_folder_path+'*Scan 1.txt'):
    dose = int(file_path.split('-')[2].split(' ')[0])

    # open the text file and create a list of all rows except the first 6 
    f = open(file_path)
    all_data = f.readlines()[6:]

    # create a list of all data with headings
    data_array = list([['Data Point', 'Value [G]', 'Intensity']])
    
    for row in all_data:
        Data_Point, Value_G, Intensity = list(filter(None, row.split(' ')))
        data_array += list([[int(Data_Point), float(Value_G), float(Intensity)]])
    
    # take the list of all data points and divide into columns with no headings
    Values = np.array([row[1] for row in data_array[1:]])
    Intensities = np.array([row[2] for row in data_array[1:]])

    # shift the magnetic field values so that the interal standard is 3540 G when the intensity is ~0
    s_centre_index = np.argmin(abs(Intensities[s_start:s_stop]))
    shift = Values[s_start:s_stop][s_centre_index]-s_B
    Values -= shift
    
    # determine the min and max value of the peaks
    a_min = min(Intensities[a_start:a_stop])
    a_max = max(Intensities[a_start:a_stop])
    s_min = min(Intensities[s_start:s_stop])
    s_max = max(Intensities[s_start:s_stop])
    
    # plot the data
    fig, ax = plt.subplots(1, figsize=(6,4))
    ax.plot(Values, Intensities, 'b', alpha=0.3)
    ax.plot(Values[a_start:a_stop], Intensities[a_start:a_stop], 'b')
    ax.plot(Values[s_start:s_stop], Intensities[s_start:s_stop], 'b') 
    ax.hlines(a_max, Values[a_start], Values[a_start]+bar_width, 'k')
    ax.hlines(a_min, Values[a_stop]-bar_width, Values[a_stop], 'k')
    ax.hlines(s_max, Values[s_start], Values[s_start]+bar_width, 'k')
    ax.hlines(s_min, Values[s_stop]-bar_width, Values[s_stop], 'k')
    ax.set(xlabel='Magnetic Field [G]', ylabel='Intensity')
    ax.set_title(r'{} Gy'.format(dose))
    ax.grid()
    ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useOffset=False, useMathText=True))

This cell below determines the P2P amplitude of the group 1 dosimeters and stores the amplitudes in a Pandas dataframe. The amplitudes are then averaged and normalized and the error is calculated. 

As before, line 4 may need to be changed with a different text file naming convention. 

In [ ]:
group1_df = pd.DataFrame()

for file_path in glob.glob(group1_folder_path+'*.txt'):
    dose, scan_number = int(file_path.split('-')[2].split(' ')[0]), file_path.split('-')[-1].split('.')[0]
    
    # use the file path to open the text file
    f = open(file_path)
    all_data = f.readlines()[6:]
    
    # use the text file to create a 2D list of the data
    data_array = []
    for row in all_data:
        Data_Point, Value_G, Intensity = list(filter(None, row.split(' ')))
        data_array += list([[int(Data_Point), float(Value_G), float(Intensity)]])

    # create np arrays from the 2D list of the data
    Values = np.array([row[1] for row in data_array[1:]])
    Intensities = np.array([row[2] for row in data_array[1:]])
    
    # get the alanine center peak and internal standard peak min and max
    a_min = min(Intensities[a_start:a_stop])
    a_max = max(Intensities[a_start:a_stop])
    s_min = min(Intensities[s_start:s_stop])
    s_max = max(Intensities[s_start:s_stop])

    # add the P2P amplitudes to the column (based on peak and scan number) and index by dose
    group1_df.loc[dose, 'Dose [Gy]'] = dose
    group1_df.loc[dose, 'A P2P '+scan_number] = a_max-a_min
    group1_df.loc[dose, 'S P2P '+scan_number] = s_max-s_min
    
    f.close()

# calculate the averages/STD
group1_df['A Average'] = group1_df[[column for column in group1_df.columns if column.startswith('A')]].mean(axis=1)
group1_df['A STD'] = group1_df[[column for column in group1_df.columns if column.startswith('A')]].std(axis=1)
group1_df['S Average'] = group1_df[[column for column in group1_df.columns if column.startswith('S')]].mean(axis=1)
group1_df['S STD'] = group1_df[[column for column in group1_df.columns if column.startswith('S')]].std(axis=1)

# calculate the density and density error
group1_df['Density'] = (4*group1_mass)/(np.pi*group1_fill_height*4**2)
group1_df['Density Error'] = group1_df['Density']*((height_err/group1_fill_height)**2+(mass_err/group1_mass)**2)**0.5

# calculate the normalized P2P amplitudes
group1_df['Standard Normalized P2P'] = group1_df['A Average']/group1_df['S Average']
group1_df['Density Normalized P2P'] = group1_df['A Average']/group1_df['Density']
group1_df['Standard+Density Normalized P2P'] = group1_df['Density Normalized P2P']/group1_df['S Average']

# calculate the error in the normalized amplitudes using error propagation
group1_df['DN P2P err'] = group1_df['Density Normalized P2P']*((group1_df['Density Error']/group1_df['Density'])+(group1_df['A STD']/group1_df['A Average']))
group1_df['SN P2P err'] = group1_df['Standard Normalized P2P']*((group1_df['A STD']/group1_df['A Average'])+(group1_df['S STD']/group1_df['S Average']))
group1_df['DN+SN P2P err'] = group1_df['Standard+Density Normalized P2P']*((group1_df['Density Error']/group1_df['Density'])+(group1_df['A STD']/group1_df['A Average'])+(group1_df['S STD']/group1_df['S Average']))

# display the final datafram
group1_df

Next, linear regression is performed and the calibration curves are plotted. The not normalized, internal standard normalized, density normalized, and the density and internal standard normalized regression results are denoted 1-4 respectively. 

In [ ]:
result1 = stats.linregress(group1_df['Dose [Gy]'], group1_df['A Average'])
result2 = stats.linregress(group1_df['Dose [Gy]'], group1_df['Standard Normalized P2P']) 
result3 = stats.linregress(group1_df['Dose [Gy]'], group1_df['Density Normalized P2P'])
result4 = stats.linregress(group1_df['Dose [Gy]'], group1_df['Standard+Density Normalized P2P'])

fig, axs = plt.subplots(2, 2, figsize=(10,8))

# not normalized
axs[0, 0].set_title('P2P Amplitude')
axs[0, 0].plot(group1_df['Dose [Gy]'], result1.slope*group1_df['Dose [Gy]']+result1.intercept, 'k--', alpha=0.4)
axs[0, 0].errorbar(group1_df['Dose [Gy]'], group1_df['A Average'], xerr=0, yerr=group1_df['A STD'], capsize=3, linestyle='none', color='k')
axs[0, 0].plot(group1_df['Dose [Gy]'], group1_df['A Average'], 'ko', markersize=5, ls='none')

# internal standard normalized 
axs[0, 1].set_title("Internal Standard Normalized")
axs[0, 1].plot(group1_df['Dose [Gy]'], result2.slope*group1_df['Dose [Gy]']+result2.intercept, 'k--', alpha=0.4)
axs[0, 1].errorbar(group1_df['Dose [Gy]'], group1_df['Standard Normalized P2P'], xerr=0, yerr=group1_df['SN P2P err'], capsize=3, linestyle='none', color='k')
axs[0, 1].plot(group1_df['Dose [Gy]'], group1_df['Standard Normalized P2P'], 'ko', markersize=5, ls='none')

# density normalized
axs[1, 0].set_title("Density Normalized")
axs[1, 0].plot(group1_df['Dose [Gy]'], result3.slope*group1_df['Dose [Gy]']+result3.intercept, 'k--', alpha=0.4)
axs[1, 0].errorbar(group1_df['Dose [Gy]'], group1_df['Density Normalized P2P'], xerr=0, yerr=group1_df['DN P2P err'], capsize=3, linestyle='none', color='k')
axs[1, 0].plot(group1_df['Dose [Gy]'], group1_df['Density Normalized P2P'], 'ko', markersize=5, ls='none')

# internal standard and density normalized
axs[1, 1].set_title("Internal Standard and Density Normalized")
axs[1, 1].plot(group1_df['Dose [Gy]'], result4.slope*group1_df['Dose [Gy]']+result4.intercept, 'k--', alpha=0.4)
axs[1, 1].errorbar(group1_df['Dose [Gy]'], group1_df['Standard+Density Normalized P2P'], xerr=0, yerr=group1_df['DN+SN P2P err'], capsize=3, linestyle='none', color='k')
axs[1, 1].plot(group1_df['Dose [Gy]'], group1_df['Standard+Density Normalized P2P'], 'ko', markersize=5, ls='none')

# formatting for all plots
for ax in axs.flat:
    ax.set(ylabel='P2P Amplitude (arb. unit)', xlabel='Dose (Gy)')
    ax.grid(alpha=0.5)
fig.tight_layout()

This cell imports the group 2 dosimeters as before. After the amplitudes are determined, the doses are calculated using the regression results from the group 1 dosimeters. As before, the not normalized, internal standard normalized, density normalized, and the density and internal standard normalized results are denoted 1-4 respectively. 

In [ ]:
# get the slopes, intercepts, and errors from regression results 
m1, m_err1, b1, b_err1 = result1.slope, result1.stderr, result1.intercept, result1.intercept_stderr # not normalized
m2, m_err2, b2, b_err2 = result2.slope, result2.stderr, result2.intercept, result2.intercept_stderr # IS normalized
m3, m_err3, b3, b_err3 = result3.slope, result3.stderr, result3.intercept, result3.intercept_stderr # density normalized
m4, m_err4, b4, b_err4 = result4.slope, result4.stderr, result4.intercept, result4.intercept_stderr # IS+density normalized 

group2_df = pd.DataFrame()

for file_path in glob.glob(group2_folder_path+'*.txt'):
    dose, scan_number = int(file_path.split('-')[2].split(' ')[0]), file_path.split('-')[-1].split('.')[0]
    
    # use the file path to open the text file
    f = open(file_path)
    all_data = f.readlines()[6:]
    
    # use the text file to create a 2D list of the data
    data_array = []
    for row in all_data:
        Data_Point, Value_G, Intensity = list(filter(None, row.split(' ')))
        data_array += list([[int(Data_Point), float(Value_G), float(Intensity)]])

    # create np arrays from the 2D list of the data
    Values = np.array([row[1] for row in data_array[1:]])
    Intensities = np.array([row[2] for row in data_array[1:]])
    
    # get the alanine center peak and internal standard peak min and max
    a_min = min(Intensities[a_start:a_stop])
    a_max = max(Intensities[a_start:a_stop])
    s_min = min(Intensities[s_start:s_stop])
    s_max = max(Intensities[s_start:s_stop])

    # plot the data to verify the peaks were identified correctly
    if file_path.endswith('Scan 1.txt'):
        fig, ax = plt.subplots(1, figsize=(6,4))
        ax.plot(Values, Intensities, 'b', alpha=0.3)
        ax.plot(Values[a_start:a_stop], Intensities[a_start:a_stop], 'b')
        ax.plot(Values[s_start:s_stop], Intensities[s_start:s_stop], 'b') 
        ax.hlines(a_max, Values[a_start], Values[a_start]+bar_width, 'k')
        ax.hlines(a_min, Values[a_stop]-bar_width, Values[a_stop], 'k')
        ax.hlines(s_max, Values[s_start], Values[s_start]+bar_width, 'k')
        ax.hlines(s_min, Values[s_stop]-bar_width, Values[s_stop], 'k')
        ax.set(xlabel='Magnetic Field [G]', ylabel='Intensity')
        ax.set_title(r'{} Gy'.format(dose))
        ax.grid()
        ax.yaxis.set_major_formatter(mticker.ScalarFormatter(useOffset=False, useMathText=True))

    # add the P2P amplitudes to the column (based on peak and scan number) and index by dose
    group2_df.loc[dose, 'Expected Dose [Gy]'] = dose
    group2_df.loc[dose, 'A P2P '+scan_number] = a_max-a_min
    group2_df.loc[dose, 'S P2P '+scan_number] = s_max-s_min
    
    f.close()

# calculate the averages/STD
group2_df['A Average'] = group2_df[[column for column in group2_df.columns if column.startswith('A')]].mean(axis=1)
group2_df['A STD'] = group2_df[[column for column in group2_df.columns if column.startswith('A')]].std(axis=1)
group2_df['S Average'] = group2_df[[column for column in group2_df.columns if column.startswith('S')]].mean(axis=1)
group2_df['S STD'] = group2_df[[column for column in group2_df.columns if column.startswith('S')]].std(axis=1)

# calculate the density and density error
group2_df['Density'] = (4*group2_mass)/(np.pi*group2_fill_height*4**2)
group2_df['Density Error'] = group2_df['Density']*((height_err/group2_fill_height)**2+(mass_err/group2_mass)**2)**0.5

# calculate the normalized P2P amplitudes
group2_df['Standard Normalized P2P'] = group2_df['A Average']/group2_df['S Average']
group2_df['Density Normalized P2P'] = group2_df['A Average']/group2_df['Density']
group2_df['Standard+Density Normalized P2P'] = group2_df['Density Normalized P2P']/group2_df['S Average']

# calculate the error in the normalized amplitudes using error propagation
group2_df['DN P2P err'] = group2_df['Density Normalized P2P']*((group2_df['Density Error']/group2_df['Density'])+(group2_df['A STD']/group2_df['A Average']))
group2_df['SN P2P err'] = group2_df['Standard Normalized P2P']*((group2_df['A STD']/group2_df['A Average'])+(group2_df['S STD']/group2_df['S Average']))
group2_df['DN+SN P2P err'] = group2_df['Standard+Density Normalized P2P']*((group2_df['Density Error']/group2_df['Density'])+(group2_df['A STD']/group2_df['A Average'])+(group2_df['S STD']/group2_df['S Average']))

# use the slope and intercept to calculate the dose and error on the dose
group2_df['Dose'] = (group2_df['A Average']-b1)/m1
group2_df['Dose Error'] = group2_df['Dose']*(((group2_df['A STD']+b_err1)/(group2_df['A Average']-b1))+(m_err1/m1))
group2_df['Dose % Error'] = abs(group2_df['Dose']-group2_df['Expected Dose [Gy]'])/group2_df['Expected Dose [Gy]']*100

group2_df['SN Dose'] = (group2_df['Standard Normalized P2P']-b2)/m2
group2_df['SN Dose Error'] = group2_df['SN Dose']*(((group2_df['SN P2P err']+b_err2)/(group2_df['Standard Normalized P2P']-b2))+(m_err2/m2))
group2_df['SN Dose % Error'] = abs(group2_df['SN Dose']-group2_df['Expected Dose [Gy]'])/group2_df['Expected Dose [Gy]']*100

group2_df['DN Dose'] = (group2_df['Density Normalized P2P']-b3)/m3
group2_df['DN Dose Error'] = group2_df['DN Dose']*(((group2_df['DN P2P err']+b_err3)/(group2_df['Density Normalized P2P']-b3))+(m_err3/m3))
group2_df['DN Dose % Error'] = abs(group2_df['DN Dose']-group2_df['Expected Dose [Gy]'])/group2_df['Expected Dose [Gy]']*100

group2_df['SN+DN Dose'] = (group2_df['Standard+Density Normalized P2P']-b4)/m4
group2_df['SN+DN Dose Error'] = group2_df['SN+DN Dose']*(((group2_df['DN+SN P2P err']+b_err4)/(group2_df['Standard+Density Normalized P2P']-b4))+(m_err4/m4))
group2_df['SN+DN Dose % Error'] = abs(group2_df['SN+DN Dose']-group2_df['Expected Dose [Gy]'])/group2_df['Expected Dose [Gy]']*100

# display the dose calculations and error
group2_df.iloc[:,23:]